In [1]:
# Copyright 2018 The TensorFlow Authors.
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#https://github.com/tensorflow/docs/blob/master/site/en/tutorials/sequences/text_generation.ipynb

In [3]:
import tensorflow as tf
import numpy as np
import os
import time
tf.autograph.set_verbosity(3, True)

In [4]:
file='1400-0.txt'
url='https://www.gutenberg.org/files/1400/1400-0.txt' # Great Expectations by Charles Dickens


In [5]:
path = tf.keras.utils.get_file(file,url)

In [6]:
text = open(path).read()
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1014396 characters


In [7]:
# strip off text we don't need
text = text[835:]

# Take a look at the first 300 characters in text
print(text[:300])






Great Expectations

[1867 Edition]

by Charles Dickens


Contents

 Chapter I.
 Chapter II.
 Chapter III.
 Chapter IV.
 Chapter V.
 Chapter VI.
 Chapter VII.
 Chapter VIII.
 Chapter IX.
 Chapter X.
 Chapter XI.
 Chapter XII.
 Chapter XIII.
 Chapter XIV.
 Chapter XV.
 Chapter XVI.
 Chapter XVII.


In [8]:
# The unique characters in the file
vocabulary = sorted(set(text))
print ('{} unique characters.'.format(len(vocabulary)))

92 unique characters.


In [9]:
vocabulary_size = len(vocabulary)

In [10]:
# Creating a dictionary of unique characters to indices
char_to_index = {char:index for index, char in enumerate(vocabulary)}
print(char_to_index)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, '@': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, ']': 57, '_': 58, 'a': 59, 'b': 60, 'c': 61, 'd': 62, 'e': 63, 'f': 64, 'g': 65, 'h': 66, 'i': 67, 'j': 68, 'k': 69, 'l': 70, 'm': 71, 'n': 72, 'o': 73, 'p': 74, 'q': 75, 'r': 76, 's': 77, 't': 78, 'u': 79, 'v': 80, 'w': 81, 'x': 82, 'y': 83, 'z': 84, 'ê': 85, 'ô': 86, '—': 87, '‘': 88, '’': 89, '“': 90, '”': 91}


In [11]:
index_to_char = np.array(vocabulary)
print(index_to_char)
text_as_int = np.array([char_to_index[char] for char in text])

['\t' '\n' ' ' '!' '"' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.' '/' '0' '1'
 '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '?' '@' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X'
 'Y' 'Z' '[' ']' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm'
 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' 'ê' 'ô' '—' '‘' '’'
 '“' '”']


In [12]:
print('{')
for char,_ in zip(char_to_index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_index[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  ...
}


In [13]:
# Show how the first 15 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:15]), text_as_int[:15]))

'\n\n\n\n\nGreat Expe' ---- characters mapped to int ---- > [ 1  1  1  1  1 36 76 63 59 78  2 34 82 74 63]


In [14]:
# The maximum length sentence we want for a single input in characters
sequence_length = 100
examples_per_epoch = len(text)//sequence_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for char in char_dataset.take(5):
  print(index_to_char[char.numpy()])

In [15]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index_to_char[item.numpy()])))

'\n\n\n\n\nGreat Expectations\n\n[1867 Edition]\n\nby Charles Dickens\n\n\nContents\n\n Chapter I.\n Chapter II.\n Cha'
'pter III.\n Chapter IV.\n Chapter V.\n Chapter VI.\n Chapter VII.\n Chapter VIII.\n Chapter IX.\n Chapter X.'
'\n Chapter XI.\n Chapter XII.\n Chapter XIII.\n Chapter XIV.\n Chapter XV.\n Chapter XVI.\n Chapter XVII.\n C'
'hapter XVIII.\n Chapter XIX.\n Chapter XX.\n Chapter XXI.\n Chapter XXII.\n Chapter XXIII.\n Chapter XXIV.\n'
' Chapter XXV.\n Chapter XXVI.\n Chapter XXVII.\n Chapter XXVIII.\n Chapter XXIX.\n Chapter XXX.\n Chapter X'


In [16]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

INFO:tensorflow:Converted call: <function split_input_target at 0x7f34299bd040>
    args: (<tf.Tensor 'args_0:0' shape=(101,) dtype=int64>,)
    kwargs: {}

Converted call: <function split_input_target at 0x7f34299bd040>
    args: (<tf.Tensor 'args_0:0' shape=(101,) dtype=int64>,)
    kwargs: {}

INFO:tensorflow:Not allowed: <method-wrapper '__call__' of function object at 0x7f34299bd040>: default rule
Not allowed: <method-wrapper '__call__' of function object at 0x7f34299bd040>: default rule
INFO:tensorflow:Not allowed: <function split_input_target at 0x7f34299bd040>: default rule
Not allowed: <function split_input_target at 0x7f34299bd040>: default rule
INFO:tensorflow:<function split_input_target at 0x7f34299bd040> is not cached for subkey ConversionOptions[{}]
<function split_input_target at 0x7f34299bd040> is not cached for subkey ConversionOptions[{}]
INFO:tensorflow:Source code of <function split_input_target at 0x7f34299bd040>:

def split_input_target(chunk):
    input_text = c

In [17]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(index_to_char[input_example.numpy()])))
  print ('Target data:', repr(''.join(index_to_char[target_example.numpy()])))

Input data:  '\n\n\n\n\nGreat Expectations\n\n[1867 Edition]\n\nby Charles Dickens\n\n\nContents\n\n Chapter I.\n Chapter II.\n Ch'
Target data: '\n\n\n\nGreat Expectations\n\n[1867 Edition]\n\nby Charles Dickens\n\n\nContents\n\n Chapter I.\n Chapter II.\n Cha'


In [18]:
for char, (input_index, target_index) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(char))
    print("  input: {} ({:s})".format(input_index, repr(index_to_char[input_index])))
    print("  expected output: {} ({:s})".format(target_index, repr(index_to_char[target_index])))

Step    0
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    1
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    2
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    3
  input: 1 ('\n')
  expected output: 1 ('\n')
Step    4
  input: 1 ('\n')
  expected output: 36 ('G')


In [19]:

# Batch size
batch = 64
steps_per_epoch = examples_per_epoch//batch

# TF data maintains a buffer in memory in which to shuffle data
# since it is designed to work with possibly endless data
buffer = 10000

dataset = dataset.shuffle(buffer).batch(batch, drop_remainder=True)

dataset = dataset.repeat()

dataset

<RepeatDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [20]:
#  The vocabulary length in characterrs
vocabulary_length = len(vocabulary)

# The embedding dimension
embedding_dimension = 256

# Number of RNN units
recurrent_nn_units = 1024

In [21]:
if tf.test.is_gpu_available():
    recurrent_nn = tf.compat.v1.keras.layers.CuDNNGRU
    print("GPU in use")
else:
    import functools
    recurrent_nn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
    print("CPU in use")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU in use


In [22]:

def build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size):
    model = tf.keras.Sequential(
        [tf.keras.layers.Embedding(vocabulary_size, embedding_dimension, batch_input_shape=[batch_size, None]),
    recurrent_nn(recurrent_nn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    tf.keras.layers.Dense(vocabulary_length)
  ])
    return model

In [23]:
model = build_model(
  vocabulary_size = len(vocabulary),
  embedding_dimension=embedding_dimension,
  recurrent_nn_units=recurrent_nn_units,
  batch_size=batch)

In [24]:
for batch_input_example, batch_target_example in dataset.take(1):
    batch_predictions_example = model(batch_input_example)
    print(batch_predictions_example.shape, "# (batch, sequence_length, vocabulary_length)")

(64, 100, 92) # (batch, sequence_length, vocabulary_length)


In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23552     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 92)            94300     
Total params: 4,056,156
Trainable params: 4,056,156
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(logits=batch_predictions_example[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
sampled_indices

array([ 4, 84, 47, 14, 35, 37, 52, 88, 73, 53, 59, 56, 70, 74, 32, 19, 48,
       11, 51, 58, 10, 43, 22, 73, 45, 84, 86, 72,  9, 79,  6, 20, 76,  7,
       44, 63, 21, 49, 89, 11, 11, 66, 57, 75, 47, 65, 91, 39,  6, 24, 34,
       90,  6, 18, 28, 84, 43, 68, 81, 89, 62, 67, 62, 36, 26, 73, 85, 28,
       25, 69,  4, 86, 46, 62, 38,  8,  7, 41, 91, 54, 30, 49, 70, 79, 18,
       83, 14,  9, 27,  2, 57, 78, 10, 16, 34, 20, 19, 30, 40, 45])

In [28]:
print("Input: \n", repr("".join(index_to_char[batch_input_example[0]])))

print("Next Char Predictions: \n", repr("".join(index_to_char[sampled_indices ])))
#

Input: 
 'sleeves, “I\nhave probably done the most I can do; but if I can ever do more,—from a\nWalworth point o'
Next Char Predictions: 
 '"zR.FHW‘oXa[lpC3S*V_)N6oPzôn(u%4r&Oe5T’**h]qRg”J%8E“%2?zNjw’didG:oê?9k"ôQdI\'&L”YATlu2y.(; ]t)0E43AKP'


In [29]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [30]:

batch_loss_example  = tf.compat.v1.losses.sparse_softmax_cross_entropy(batch_target_example, batch_predictions_example)
print("Prediction shape: ", batch_predictions_example.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", batch_loss_example.numpy())

Prediction shape:  (64, 100, 92)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.523453


In [31]:
#next produced by upgrade script.... 
#model.compile(optimizer = tf.compat.v1.train.AdamOptimizer(), loss = loss) 
#.... but following optimizer is available.
model.compile(optimizer = tf.optimizers.Adam(), loss = loss)

In [32]:
# Directory where the checkpoints will be saved
directory = './checkpoints'
# Name of the checkpoint files
file_prefix = os.path.join(directory, "ckpt_{epoch}")

callback=[tf.keras.callbacks.ModelCheckpoint(filepath=file_prefix, save_weights_only=True)]


In [33]:
epochs=45

In [34]:

history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=callback)

Epoch 1/45
INFO:tensorflow:Converted call: <function Model.make_train_function.<locals>.train_function at 0x7f3433de25e0>
    args: (<tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x7f342a17d910>,)
    kwargs: {}

Converted call: <function Model.make_train_function.<locals>.train_function at 0x7f3433de25e0>
    args: (<tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x7f342a17d910>,)
    kwargs: {}

INFO:tensorflow:<function Model.make_train_function.<locals>.train_function at 0x7f3433de25e0> is not cached for subkey ConversionOptions[{}]
<function Model.make_train_function.<locals>.train_function at 0x7f3433de25e0> is not cached for subkey ConversionOptions[{}]
INFO:tensorflow:Source code of <function Model.make_train_function.<locals>.train_function at 0x7f3433de25e0>:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
def train_function(iterator):
  """Runs a training execution with one step

Transformed <function loss at 0x7f3428c21550>:

# coding=utf-8
def tf__loss(labels, logits):
    with ag__.FunctionScope('loss', 'fscope', ag__.STD) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = ag__.converted_call(ag__.ld(tf).keras.losses.sparse_categorical_crossentropy, (ag__.ld(labels), ag__.ld(logits)), dict(from_logits=True), fscope)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

INFO:tensorflow:Defaults of <function outer_factory.<locals>.inner_factory.<locals>.tf__loss at 0x7f3420469ee0> : None
Defaults of <function outer_factory.<locals>.inner_factory.<locals>.tf__loss at 0x7f3420469ee0> : None
INFO:tensorflow:KW defaults of <function outer_factory.<locals>.inner_factory.<locals>.tf__loss at 0x7f3420469ee0> : None
KW defaults of <function outer_factory.<locals>.inner_factory.<locals>.tf__loss at 0x7f3420469ee0> : No

Forwarding call of partial functools.partial(<bound method OptimizerV2._distributed_apply of <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f3428cf6bb0>>, apply_state={('/job:localhost/replica:0/task:0/device:GPU:0', tf.float32): {'lr_t': <tf.Tensor 'Adam/Identity:0' shape=() dtype=float32>, 'lr': <tf.Tensor 'Adam/mul:0' shape=() dtype=float32>, 'epsilon': <tf.Tensor 'Adam/Const:0' shape=() dtype=float32>, 'beta_1_t': <tf.Tensor 'Adam/Identity_1:0' shape=() dtype=float32>, 'beta_1_power': <tf.Tensor 'Adam/Pow:0' shape=() dtype=float32>, 'one_minus_beta_1_t': <tf.Tensor 'Adam/sub_2:0' shape=() dtype=float32>, 'beta_2_t': <tf.Tensor 'Adam/Identity_2:0' shape=() dtype=float32>, 'beta_2_power': <tf.Tensor 'Adam/Pow_1:0' shape=() dtype=float32>, 'one_minus_beta_2_t': <tf.Tensor 'Adam/sub_3:0' shape=() dtype=float32>}, ('/job:localhost/replica:0/task:0/device:CPU:0', tf.float32): {'lr_t': <tf.Tensor 'Adam/Identity_3:0' shape=() dtype=float32>, 'lr': <tf.Tensor 'Adam/mul_1:0' sh

Allowlisted: <bound method OptimizerV2._distributed_apply of <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f3428cf6bb0>>: DoNotConvert rule for tensorflow
INFO:tensorflow:Converted call: <function OptimizerV2._distributed_apply.<locals>.apply_grad_to_update_var at 0x7f3420465670>
    args: (<tf.Variable 'embedding/embeddings:0' shape=(92, 256) dtype=float32>, <tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f34203f51c0>)
    kwargs: {}

Converted call: <function OptimizerV2._distributed_apply.<locals>.apply_grad_to_update_var at 0x7f3420465670>
    args: (<tf.Variable 'embedding/embeddings:0' shape=(92, 256) dtype=float32>, <tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f34203f51c0>)
    kwargs: {}

INFO:tensorflow:Allowlisted: <function OptimizerV2._distributed_apply.<locals>.apply_grad_to_update_var at 0x7f3420465670>: DoNotConvert rule for tensorflow
Allowlisted: <function OptimizerV2._distributed_apply.<locals>.apply_grad

Allowlisted <function Model.make_train_function.<locals>.step_function at 0x7f342a0945e0>: from cache
INFO:tensorflow:Converted call: <function Model.make_train_function.<locals>.step_function.<locals>.run_step at 0x7f3420465ca0>
    args: ((<tf.Tensor 'IteratorGetNext:0' shape=(64, 100) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(64, 100) dtype=int64>),)
    kwargs: {}

Converted call: <function Model.make_train_function.<locals>.step_function.<locals>.run_step at 0x7f3420465ca0>
    args: ((<tf.Tensor 'IteratorGetNext:0' shape=(64, 100) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(64, 100) dtype=int64>),)
    kwargs: {}

INFO:tensorflow:Allowlisted: <function Model.make_train_function.<locals>.step_function.<locals>.run_step at 0x7f3420465ca0>: DoNotConvert rule for tensorflow
Allowlisted: <function Model.make_train_function.<locals>.step_function.<locals>.run_step at 0x7f3420465ca0>: DoNotConvert rule for tensorflow
INFO:tensorflow:Converted call: <bound method LossFu

Converted call: functools.partial(<bound method OptimizerV2._distributed_apply of <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f3428cf6bb0>>, apply_state={('/job:localhost/replica:0/task:0/device:GPU:0', tf.float32): {'lr_t': <tf.Tensor 'Adam/Identity:0' shape=() dtype=float32>, 'lr': <tf.Tensor 'Adam/mul:0' shape=() dtype=float32>, 'epsilon': <tf.Tensor 'Adam/Const:0' shape=() dtype=float32>, 'beta_1_t': <tf.Tensor 'Adam/Identity_1:0' shape=() dtype=float32>, 'beta_1_power': <tf.Tensor 'Adam/Pow:0' shape=() dtype=float32>, 'one_minus_beta_1_t': <tf.Tensor 'Adam/sub_2:0' shape=() dtype=float32>, 'beta_2_t': <tf.Tensor 'Adam/Identity_2:0' shape=() dtype=float32>, 'beta_2_power': <tf.Tensor 'Adam/Pow_1:0' shape=() dtype=float32>, 'one_minus_beta_2_t': <tf.Tensor 'Adam/sub_3:0' shape=() dtype=float32>}, ('/job:localhost/replica:0/task:0/device:CPU:0', tf.float32): {'lr_t': <tf.Tensor 'Adam/Identity_3:0' shape=() dtype=float32>, 'lr': <tf.Tensor 'Adam/mul_1:0' shape=() dtyp

Converted call: <bound method OptimizerV2._distributed_apply of <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f3428cf6bb0>>
    args: (<tensorflow.python.distribute.distribute_lib._DefaultDistributionStrategy object at 0x7f3428cbfca0>, [(<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7f342035b700>, <tf.Variable 'embedding/embeddings:0' shape=(92, 256) dtype=float32>), (<tf.Tensor 'Adam/gradients/AddN:0' shape=(256, 3072) dtype=float32>, <tf.Variable 'cu_dnngru/kernel:0' shape=(256, 3072) dtype=float32>), (<tf.Tensor 'Adam/gradients/AddN_1:0' shape=(1024, 3072) dtype=float32>, <tf.Variable 'cu_dnngru/recurrent_kernel:0' shape=(1024, 3072) dtype=float32>), (<tf.Tensor 'Adam/gradients/AddN_2:0' shape=(6144,) dtype=float32>, <tf.Variable 'cu_dnngru/bias:0' shape=(6144,) dtype=float32>), (<tf.Tensor 'gradient_tape/sequential/dense/Tensordot/MatMul/MatMul_1:0' shape=(1024, 92) dtype=float32>, <tf.Variable 'dense/kernel:0' shape=(1024, 92) dtype=float32>)

Allowlisted <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7f342a187850>>: from cache
158/158 [==============================] - 4s 20ms/step - loss: 3.2360
Epoch 2/45
158/158 [==============================] - 3s 20ms/step - loss: 2.0621
Epoch 3/45
158/158 [==============================] - 3s 20ms/step - loss: 1.7342
Epoch 4/45
158/158 [==============================] - 3s 20ms/step - loss: 1.5493
Epoch 5/45
158/158 [==============================] - 3s 20ms/step - loss: 1.4344
Epoch 6/45
158/158 [==============================] - 3s 20ms/step - loss: 1.3569
Epoch 7/45
158/158 [==============================] - 3s 20ms/step - loss: 1.3015
Epoch 8/45
158/158 [==============================] - 3s 19ms/step - loss: 1.2573
Epoch 9/45
158/158 [==============================] - 3s 19ms/step - loss: 1.2184
Epoch 10/45
158/158 [==============================] - 3s 19ms/step - loss: 1.1869
Epoch 11/45
158/158 [==============================] - 3s 19ms/step - 

In [34]:
tf.train.latest_checkpoint(directory)

'./checkpoints/ckpt_45'

In [35]:
model = build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(directory))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
unified_gru_1 (UnifiedGRU)   (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [37]:

def generate_text(model, start_string, temperature, characters_to_generate):

  # Vectorise  start string into numbers
    input_string = [char_to_index[char] for char in start_string]
    input_string = tf.expand_dims(input_string, 0)

  # Empty string to store  generated text
    generated = []

  # (Batch size is 1)
    model.reset_states()
    for i in range(characters_to_generate):
        predictions = model(input_string)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(logits=predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_string = tf.expand_dims([predicted_id], 0)

        generated.append(index_to_char[predicted_id])

    return (start_string + ''.join(generated)) # generated is a list

In [38]:
# In the arguments, a low temperature gives more predictable text whereas a high temperature gives more random text.
# Also this is where you can change the start string.
generated_text = generate_text(model=model, start_string="Pip", temperature=0.1, characters_to_generate = 1000)
print(generated_text)

Pip!”

“So it was.”

“Astonishing!” said Joe, in the nature of an umbrella.

“Then, as it were now as for the other convict. “The fear of our own little nor
stones of the forge. I was falling into my head to look if the way by which I had come to be a man. A deserting your mind to him, my indertrodic work in any convicts!” Then both very well. I
thought it would have been more crack with the forge and Mill Pond Bank, Clara
was not a variety of being in common black piece of paper, and put the two convicts were
bought, the more certain I am of a circle.

“Lookee here, Pip, look at his door.

In the evening there was a lady by which I was always creeping the fire at the windows of the copyright holder), the waiter reappeared.

“Why you see, old chap. They'll do you suppose the attempt to do it
distinctly thanked him and said that Mr. Pumblechook was on my shoulder by some one
of these days, and as Mr. Pumblechook was not at all likely he could
have done it. I had done it, under the silen